## **Load libraries and data**

In [ ]:
!pip install --upgrade pythainlp
!pip install pyLDAvis
!pip install -U pandas-profiling
!pip install sefr_cut

Requirement already up-to-date: pythainlp in /usr/local/lib/python3.7/dist-packages (2.3.1)
Requirement already up-to-date: pandas-profiling in /usr/local/lib/python3.7/dist-packages (3.0.0)


In [ ]:
!pip install openpyxl
!pip install pandas

In [1]:
import pandas as pd
import pythainlp
import sefr_cut
import gensim
# import pyLDAvis.gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from pandas import ExcelWriter
from pandas import ExcelFile
import io
import numpy as np


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:169: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not 

In [2]:
%cd "/content/drive/My Drive/BADS7105"

/content/drive/My Drive/BADS7105


# **Load data**

In [3]:
df = pd.read_csv('CustomerReviews.csv')

In [4]:
df.head(10)

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating
0,1,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,7b16469831074f7abc7824745ee75212,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,5.0
1,2,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,pakkaramonpondej,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่งดี,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,5.0
2,3,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,saanowy,ชาบูพรีเมี่ยมสไตล์ญี่ปุ่น เนื้อดีมากกก,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,4.0
3,4,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ployynp,เนื้อดี ไอติมดี คุ้มค่าดี,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,4.0
4,5,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,665a902b335b434ab489f00f2fbb477e,อาหารมีคุณภาพ บริการดีค่ะ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,5.0
5,6,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,12f2b56b28bb4849a92690cedc1cc50a,กินให้อิ่ม ที่ momo,ร้านบุฟเฟ่ ชาบูแนวญี่ปุ่น สายเนื้อหมู เนื้อวัว...,4.0
6,7,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,f2628bf47a394345b133fec76a79661d,ชาบู - สุกกี้สไลต์ญี่ปุ่นที่แท้ทรู,Number 20 : โมโม – พาราไดส์ (สาขาเดอะมอลบางกะป...,4.0
7,8,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,backscrash,ร้านชาบูเฟรนไชส์ ที่ราคาเหมาะสมกับคุณภาพ ถือว่...,ร้านชาบูเฟรนไชส์รสชาติดีมากคุ้มค่าเหมาะสมกับรา...,4.0
8,9,4621tm-shabushi-เดอะมอลล์บางกะปิ-ชั้น-g,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,a477a5812ada4d5b80fd04fca22bdbca,อร่อยคุ้ม ของทานเล่นเยอะมาก,มา านที่ขาบูชิต้องมาตอนหิว ไม่งั้นจะไม่คุ้มนะค...,4.0
9,10,4621tm-shabushi-เดอะมอลล์บางกะปิ-ชั้น-g,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,fd2ba94ec1bc4948ae109e2cb6c9cbc6,ทานอีก ชอบกุ้งทอดเทมปุระ,ใครชอบกุ้งทอดเทมปุระ แค่กุ้งเทมปุระก็คุ้มแล้ว ...,3.0


In [5]:
df.tail()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating
16,17,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,ployynp,บุฟเฟ่ต์ชาบูและพิซซ่าไม่อั้นในราคา 199 บาท เน้...,หลังจากที่เคยลองสาขายูเนี่ยนมอลล์ไป รอบนี้มาที...,4.0
17,18,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,27a91236fe5e4559a4f097c97a480781,ร้านบุฟเฟ่ต์ ราคามิตรภาพ อยู่ชั้น4 ติดโรงหนัง ...,ร้านบุฟเฟ่ต์ที่มีโปรโมชั่นหัวละ199บาท ไม่รวมน้...,4.0
18,19,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,0b81d251e4db486f9bcdba73b374ed99,ของหลากหลาย ปนๆ งงๆ นิดหน่อย,เคยรู้จักร้านนี้จากที่ union mall ไม่เคยได้ลอง...,3.0
19,20,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,40e0e087f3914fd49a8933b5a29936ca,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,5.0
20,21,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,41841cb99ea243a3a8d4b006e946c586,แม้จะแปลกบ้าง แต่ก็ถือว่าอยู่ในเกณฑ์ที่ดี มีอา...,ก็ตามที่เขียนเลยครับ ว่า ถ้าจะจ่ายในราคา 199 บ...,NaN


## **Tokenize Words** - Text Processing

In [6]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = [' ', '  ', '\n', 'ร้าน', '(', ')','!!!!','+','++','😆','🤣','/','-','ววว','xx','อิอิ','แย้ววว','นะคะ','นะค่ะ','(',')','ฯ','ๆ','กก','ขาบูชิ','555',':', '::','แตา','ก์']
screening_words = stopwords + removed_words

import pythainlp
from pythainlp.corpus.common import thai_words

def tokenize_with_space(sentence):
  merge_name = set(thai_words())
  merge_name.add('ชาบูชิ')
  merge_name.add('ยูเนี่ยนมอลล์')
  merge_name.add('รีฟีล')
  merge_name.add('บุพเฟ่')
  merge_name.add('ปรกติ')
  merge_name.add('เทมปุระ')
  merge_name.add('หอมมาก')
  merge_name.add('บุฟเฟ่')
  merge_name.add('บุฟเฟ')
  merge_name.add('บุฟเฟ่ต์')
  merge_name.add('เดอะมอล')
  merge_name.add('เดอะมอลล์')
  merge_name.add('โมจิ')
  merge_name.add('โมโม่')
  merge_name.add('พาราไดส์')
  merge_name.add('พาราไดซ์')


  trie = pythainlp.util.dict_trie(dict_source = merge_name)
def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening_words:
      merged = merged + ',' + word.strip()
  return merged[1:]

In [7]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [8]:
df.tail()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating,Review_tokenized
16,17,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,ployynp,บุฟเฟ่ต์ชาบูและพิซซ่าไม่อั้นในราคา 199 บาท เน้...,หลังจากที่เคยลองสาขายูเนี่ยนมอลล์ไป รอบนี้มาที...,4.0,"หลังจากที่,ลอง,สาขา,ยู,นม,อลล์,รอบ,สาขา,เดอะ,ม..."
17,18,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,27a91236fe5e4559a4f097c97a480781,ร้านบุฟเฟ่ต์ ราคามิตรภาพ อยู่ชั้น4 ติดโรงหนัง ...,ร้านบุฟเฟ่ต์ที่มีโปรโมชั่นหัวละ199บาท ไม่รวมน้...,4.0,"บุฟเฟ่ต์,โปรโมชั่น,หัว,199,บาท,น้ำ,VAT,ทาน,ธรร..."
18,19,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,0b81d251e4db486f9bcdba73b374ed99,ของหลากหลาย ปนๆ งงๆ นิดหน่อย,เคยรู้จักร้านนี้จากที่ union mall ไม่เคยได้ลอง...,3.0,"รู้จัก,union,mall,ลอง,กิน,จำได้,ขึ้นใจ,ชื่อ,ชื..."
19,20,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,40e0e087f3914fd49a8933b5a29936ca,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,5.0,"อร่อย,คุ้มค่า,สมราคา,บุ,ฟเฟ่,หมู,ผัก,น้ำ,จบ,25..."
20,21,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,41841cb99ea243a3a8d4b006e946c586,แม้จะแปลกบ้าง แต่ก็ถือว่าอยู่ในเกณฑ์ที่ดี มีอา...,ก็ตามที่เขียนเลยครับ ว่า ถ้าจะจ่ายในราคา 199 บ...,NaN,"จ่าย,ราคา,199,บาท,จ่าย,เงินสด,ราคา,น้ำ,VAT,7,%..."


## **Create Dictionary**

In [9]:
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [10]:
print(dictionary.token2id.keys())

dict_keys(['1', '4', '560', 'กิน', 'ข้างนอก', 'ความต้องการ', 'คอ', 'คุณภาพ', 'คุ้ม', 'จำกัด', 'ซุป', 'ดำ', 'ดี', 'ดีมาก', 'ถาด', 'ถาม', 'น่ารัก', 'น้ำ', 'น้ำจิ้ม', 'บริการ', 'บาท', 'ประทับใจ', 'พนักงาน', 'พอกับ', 'ภาพ', 'รสชาติ', 'ราคา', 'สรุป', 'สะอาดสะอ้าน', 'สัน', 'หลากหลาย', 'อร่อย', 'อันนี้', 'อาหาร', 'เคลม', 'เซ็ท', 'เต้าเจี้ยว', 'เนื้อ', 'เป็ด', 'เผ็ด', 'แบบนี้', 'แปลก', 'แฟน', 'โดยรวม', 'โมจิ', 'ใส่ใจ', 'ไม่ต้อง', 'ไอติม', '\u200b', '', '21', '30', '9', 'CTW', 'Terminal', 'กา', 'การงด', 'คิดถึง', 'ชาบู', 'ญี่ปุ่น', 'ฐาน', 'ทะยอย', 'ทำงาน', 'ที่ทาง', 'นาที', 'นึกถึง', 'พนักงานบริการ', 'พระราม', 'พรีเมียม', 'พาราไดซ์', 'มาตรา', 'มีโอ', 'รายละเอียด', 'ลูกค้า', 'สมาทาน', 'สาขา', 'สู้', 'ออร์เดอร์', 'เครื่องดื่ม', 'เค้า', 'เจอ', 'เป็นกันเอง', 'เหวี่ยง', 'แจ้ง', 'โม', 'โม่', 'โอเค', '2', '600', 'กรุบ', 'กั้น', 'ของหวาน', 'คนละ', 'คาว', 'คุ', 'คุ้มค่า', 'จาน', 'จิ้ม', 'จืด', 'ฉาก', 'ชอบ', 'ชาเขียว', 'ชิ้น', 'ซอย', 'ซอส', 'ตก', 'ตะ', 'ตัก', 'ตามน้ำ', 'ต่างกัน', 'ถั่ว', 'ถ่ายรูป', 'ทาน'

In [11]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]


## **Topic Modeling**

In [42]:
num_topics = 4
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 366 ms, sys: 7.62 ms, total: 373 ms
Wall time: 374 ms


In [43]:
#pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary)
gensimvis.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.052119  0.002546       1        1  43.847461
2     -0.019496 -0.031722       2        1  25.663487
0     -0.027791  0.026576       3        1  21.467129
3     -0.004831  0.002601       4        1   9.021924, topic_info=      Term       Freq      Total Category  logprob  loglift
17     น้ำ  23.000000  23.000000  Default  30.0000  30.0000
31   อร่อย  20.000000  20.000000  Default  29.0000  29.0000
145  เลือก  17.000000  17.000000  Default  28.0000  28.0000
48       ​   7.000000   7.000000  Default  27.0000  27.0000
10     ซุป  15.000000  15.000000  Default  26.0000  26.0000
..     ...        ...        ...      ...      ...      ...
114   นุ่ม   0.689943   6.916118   Topic4  -5.1845   0.1005
122     รส   0.659083   6.227193   Topic4  -5.2303   0.1597
75    สาขา   0.760263  12.182056   Topic4  -5.0875  -0.3685
49           0.697246   9.752635   Topic4  -5.1740  -0.2326
8     คุ้ม   0.687661   9.859270   Topic4  -5.1879  -0.2574

[277 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
49        1  0.205073         
49        2  0.307609         
49        3  0.307609         
49        4  0.102536         
0         1  0.515925        1
...     ...       ...      ...
175       1  0.706274  ไอศกรีม
258       3  0.996997  ไอศครีม
48        1  0.142413        ​
48        2  0.712063        ​
48        3  0.142413        ​

[319 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1, 4])

In [44]:
def convert_to_keyword(x):
  new_x = str(unique_word[x])
  return new_x


def convert_to_topic(x):
  new_x = [i[0] for i in x]
  return new_x

**Review word in each topic**

In [45]:
list_topic = list()
list_topic.clear()
for i in range(0,num_topics):
  print('topic:',i)
  list_topic.append(set(convert_to_topic(model.show_topic(i,20))))
  print(list_topic[i])

topic: 0
{'', 'ดำ', 'เนื้อ', 'สั่ง', 'ไอติม', '2', 'เลือก', 'ทาน', 'ชอบ', 'กิน', 'ดี', 'ซุป', 'บาท', 'รสชาติ', 'อร่อย', 'ชาบู', 'อาหาร', 'น้ำจิ้ม', 'น้ำ', 'ราคา'}
topic: 1
{'กุ้ง', 'เนื้อ', 'คน', 'สด', 'ปุระ', 'สาขา', 'เลือก', 'ม', 'ทาน', 'กิน', 'ดี', 'บาท', 'ชิ', 'รสชาติ', 'อร่อย', 'ชาบู', 'อาหาร', 'เท', 'น้ำ', 'ราคา'}
topic: 2
{'', 'เนื้อ', 'สั่ง', 'สาขา', 'เลือก', 'บริการ', 'ทาน', 'กิน', 'ดี', 'ซุป', 'บาท', 'รสชาติ', 'อร่อย', 'ชาบู', 'อาหาร', 'คุ้ม', 'ลอง', 'น้ำ', '\u200b', 'ราคา'}
topic: 3
{'ของหวาน', 'เนื้อ', 'ไอติม', 'สาขา', 'เลือก', 'ทาน', 'ชอบ', 'กิน', 'ดี', 'ซุป', 'บาท', 'บุ', 'รสชาติ', 'อร่อย', 'ชาบู', 'บุฟเฟ่ต์', 'อาหาร', 'คนละ', 'น้ำ', 'ราคา'}


**Unique keyword in each topic**

In [47]:
unique_word = list()
unique_word.clear()
for i in range(0,num_topics):
  a = list_topic[i]
  b = set()
  for j in range(0,num_topics):
    if i != j:
      b = b.union(list_topic[j])
  a = a-b
  unique_word.append(list(a))

print(unique_word)

[['ดำ', '2', 'น้ำจิ้ม'], ['กุ้ง', 'เท', 'ชิ', 'คน', 'ม', 'สด', 'ปุระ'], ['\u200b', 'คุ้ม', 'ลอง', 'บริการ'], ['บุ', 'ของหวาน', 'คนละ', 'บุฟเฟ่ต์']]


In [48]:
d = list_topic[0]
print(d)
for i in range(1,num_topics):
  # print(list_topic[i])
  d = d.intersection(list_topic[i])
  print(i,d)

{'', 'ดำ', 'เนื้อ', 'สั่ง', 'ไอติม', '2', 'เลือก', 'ทาน', 'ชอบ', 'กิน', 'ดี', 'ซุป', 'บาท', 'รสชาติ', 'อร่อย', 'ชาบู', 'อาหาร', 'น้ำจิ้ม', 'น้ำ', 'ราคา'}
1 {'ดี', 'บาท', 'เนื้อ', 'เลือก', 'รสชาติ', 'อร่อย', 'ชาบู', 'น้ำ', 'อาหาร', 'ทาน', 'ราคา', 'กิน'}
2 {'ดี', 'บาท', 'เนื้อ', 'เลือก', 'รสชาติ', 'อร่อย', 'ชาบู', 'น้ำ', 'อาหาร', 'ทาน', 'ราคา', 'กิน'}
3 {'ดี', 'บาท', 'เนื้อ', 'เลือก', 'รสชาติ', 'อร่อย', 'ชาบู', 'น้ำ', 'อาหาร', 'ทาน', 'ราคา', 'กิน'}


In [49]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])
df['topic_name'] = df['Review_tokenized'].apply(lambda x: model.show_topic(model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0]))
df['topic_name_word'] = df['topic_name'].apply(lambda x:convert_to_topic(x))
df['keyword'] = df['topics'].apply(lambda x:convert_to_keyword(x))

In [50]:
df[['Restaurant', 'Review', 'topics','score','topic_name_word','keyword']]

,Restaurant,Review,topics,score,topic_name_word,keyword
0,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,2,0.998652,"[อร่อย, น้ำ, กิน, ทาน, ราคา, ​, ซุป, ชาบู, บาท...","['\u200b', 'คุ้ม', 'ลอง', 'บริการ']"
1,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,1,0.998113,"[ทาน, กิน, ดี, อาหาร, กุ้ง, เนื้อ, ชาบู, ราคา,...","['กุ้ง', 'เท', 'ชิ', 'คน', 'ม', 'สด', 'ปุระ']"
2,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,0,0.999233,"[น้ำ, ชาบู, เลือก, ทาน, อร่อย, ซุป, เนื้อ, รสช...","['ดำ', '2', 'น้ำจิ้ม']"
3,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,1,0.997260,"[ทาน, กิน, ดี, อาหาร, กุ้ง, เนื้อ, ชาบู, ราคา,...","['กุ้ง', 'เท', 'ชิ', 'คน', 'ม', 'สด', 'ปุระ']"
4,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,2,0.997476,"[อร่อย, น้ำ, กิน, ทาน, ราคา, ​, ซุป, ชาบู, บาท...","['\u200b', 'คุ้ม', 'ลอง', 'บริการ']"
5,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ร้านบุฟเฟ่ ชาบูแนวญี่ปุ่น สายเนื้อหมู เนื้อวัว...,0,0.996203,"[น้ำ, ชาบู, เลือก, ทาน, อร่อย, ซุป, เนื้อ, รสช...","['ดำ', '2', 'น้ำจิ้ม']"
6,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,Number 20 : โมโม – พาราไดส์ (สาขาเดอะมอลบางกะป...,0,0.998188,"[น้ำ, ชาบู, เลือก, ทาน, อร่อย, ซุป, เนื้อ, รสช...","['ดำ', '2', 'น้ำจิ้ม']"
7,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ร้านชาบูเฟรนไชส์รสชาติดีมากคุ้มค่าเหมาะสมกับรา...,0,0.997762,"[น้ำ, ชาบู, เลือก, ทาน, อร่อย, ซุป, เนื้อ, รสช...","['ดำ', '2', 'น้ำจิ้ม']"
8,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,มา านที่ขาบูชิต้องมาตอนหิว ไม่งั้นจะไม่คุ้มนะค...,1,0.997487,"[ทาน, กิน, ดี, อาหาร, กุ้ง, เนื้อ, ชาบู, ราคา,...","['กุ้ง', 'เท', 'ชิ', 'คน', 'ม', 'สด', 'ปุระ']"
9,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,ใครชอบกุ้งทอดเทมปุระ แค่กุ้งเทมปุระก็คุ้มแล้ว ...,1,0.998353,"[ทาน, กิน, ดี, อาหาร, กุ้ง, เนื้อ, ชาบู, ราคา,...","['กุ้ง', 'เท', 'ชิ', 'คน', 'ม', 'สด', 'ปุระ']"


**Review result**

Common word for every topic e.g. เนื้อ รสชาติ อร่อย ราคา ซุป อาหาร น้ำ

Focus on each topic that word differenc from others

Topic 0: น้ำจิ้ม น้ำดำ

Topic 1: กุ้ง เทมปุระ ซูชิ

Topic 2: คุ้ม บริการ

Topic 3: ของหวาน บุฟเฟต์